# SageMakerでsklearnを使ったEndpointの作成
sklearnで学習モデルを作成し、Endpointをデプロイする。

参考: https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/scikit_learn_iris/scikit_learn_estimator_example_with_batch_transform.ipynb

In [1]:
prefix = "DEMO-scikit-iris"

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

## データの準備

In [2]:
import boto3
import numpy as np
import pandas as pd
import os

os.makedirs("./data", exist_ok=True)

s3_client = boto3.client("s3")
s3_client.download_file(
    f"sagemaker-sample-files", "datasets/tabular/iris/iris.data", "./data/iris.csv"
)

df_iris = pd.read_csv("./data/iris.csv", header=None)
df_iris[4] = df_iris[4].map({"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2})
iris = df_iris[[4, 0, 1, 2, 3]].to_numpy()
np.savetxt("./data/iris.csv", iris, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")

In [3]:
WORK_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
)

## sklearnで学習
`sklearn_custom_ml.py` というスクリプト内でsklearnを使って実装されたモデルを読み込む。
`sklearn_custom_ml.py` では以下のパラメータの実装 (argparseで渡す)。
他にもハイパーパラメータもパラメータとして渡すことができる。

- --output-data-dir
- --model-dir
- --train

In [4]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "1.0-1" # 0.20.0, 0.23-1なども選択可能
script_path = "sklearn_custom_ml.py"

sklearn = SKLearn(
    entry_point=script_path,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={"max_leaf_nodes": 30},
)

In [ ]:
sklearn.fit({"train": train_input})

2022-07-28 23:39:56 Starting - Starting the training job...
2022-07-28 23:40:12 Starting - Preparing the instances for trainingProfilerReport-1659051596: InProgress
........

## Endpoint作成
簡易的にdeployメソッドでエンドポイントを作成している。
必要あれば、model, endpoint_configを別途作ってもよい。

In [ ]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

## Endpointの動作確認

In [ ]:
endpoint_name = predictor.endpoint_name

In [ ]:
import boto3
client = boto3.client("sagemaker-runtime")

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body="5.1,3.5,1.4,0.2\n5.7,2.6,3.5,1.0",
    ContentType='text/csv',
    Accept='application/json'
)

In [ ]:
response["Body"].read()

## Endpoint削除

In [ ]:
predictor.delete_endpoint()